# Data

* **train:** El conjunto de entrenamiento consiste en una porción de los datos de tasa de clics de Jampp durante un período de 7 días, no necesariamente ordenados cronológicamente (sin embargo, se proporciona una columna de tiempo).
* **test:** El conjunto de prueba consiste en el tráfico de Jampp del día posterior al período de entrenamiento (calculado/muestreado de la misma manera que el conjunto de entrenamiento).

# Variables

* **Label:** Variable objetivo que indica si el anuncio fue clickeado (1) o no (0).
* **action_categorical_0:** ID de la unidad de negocio. Nivel 1. Estos IDs tienen una relación de uno a muchos (un ID de primer nivel puede tener muchos de segundo nivel, pero un ID de segundo nivel puede tener solo un único ID padre de primer nivel).
* **action_categorical_1:** ID de la unidad de negocio. Nivel 2.
* **action_categorical_2:** ID de la unidad de negocio. Nivel 3.
* **action_categorical_3:** ID de la unidad de negocio. Nivel 4.
* **action_categorical_4:** ID de la unidad de negocio. Nivel 5.
* **action_categorical_5:** Una variable categórica.
* **action_categorical_6:** Una variable categórica.
* **action_categorical_7:** Una variable categórica.
* **action_list_0:** Una lista de categorías relacionadas con la acción.
* **action_list_1:** Una lista de IDs de entidades relacionadas con la acción.
* **action_list_2:** Una lista de IDs de entidades relacionadas con la acción.
* **auction_time:** El tiempo en que se realizó la subasta. El formato es tiempo UNIX (también conocido como epoch time).
* **auction_age:** La edad del usuario.
* **auction_bidfloor:** El valor mínimo que se puede ofertar.
* **auction_boolean_0:** Una variable booleana relacionada con un atributo de la subasta.
* **auction_boolean_1:** Una variable booleana relacionada con un atributo de la subasta.
* **auction_boolean_2:** Una variable booleana relacionada con un atributo de la subasta no rastreado.
* **auction_categorical_0:** Un ID de entidad relacionado con un componente de la subasta.
* **auction_categorical_1:** Un ID de entidad relacionado con un componente de la subasta.
* **auction_categorical_2:** Una variable categórica.
* **auction_categorical_3:** Una variable categórica.
* **auction_categorical_4:** Una variable categórica.
* **auction_categorical_5:** Una variable categórica.
* **auction_categorical_6:** Una variable categórica.
* **auction_categorical_7:** Un ID de entidad relacionado con un componente de la subasta.
* **auction_categorical_8:** Un ID de entidad relacionado con un componente de la subasta.
* **auction_categorical_9:** Un ID de entidad relacionado con un componente de la subasta.
* **auction_categorical_10:** Una variable categórica.
* **auction_categorical_11:** Un ID de entidad relacionado con un componente de la subasta.
* **auction_categorical_12:** Una variable categórica.
* **auction_list_0:** Una lista de categorías relacionadas con la subasta.
* **creative_categorical_0:** ID de la unidad de negocio.
* **creative_categorical_1:** Una variable categórica.
* **creative_categorical_10:** Una variable categórica.
* **creative_categorical_11:** Una variable categórica.
* **creative_categorical_12:** Una variable categórica.
* **creative_categorical_2:** Una variable categórica.
* **creative_categorical_3:** Una variable categórica.
* **creative_categorical_4:** Una variable categórica.
* **creative_categorical_5:** ID de la unidad de negocio.
* **creative_categorical_6:** Una variable categórica.
* **creative_categorical_7:** Una variable categórica.
* **creative_categorical_8:** Una variable categórica.
* **creative_categorical_9:** Una variable categórica.
* **creative_height:** La altura del creativo (en píxeles).
* **creative_width:** El ancho del creativo (en píxeles).
* **device_id:** El ID del dispositivo. Este es un identificador único o casi único para un dispositivo.
* **device_id_type:** El tipo de ID del dispositivo (existen muchos tipos y un dispositivo puede tener varios de ellos, con un ID diferente para cada tipo).
* **gender:** Género.
* **has_video:** Un valor booleano que indica si el banner contiene un video.
* **timezone_offset:** Diferencia horaria en horas basada en el país y la región de la subasta.

In [3]:
import pandas as pd
import gc
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
random_state = 43992294

In [5]:
# Load part of the train data
train_data = pd.read_csv("data/ctr_21.csv")

# Load the test data
eval_data = pd.read_csv("data/ctr_test.csv")

# Train a tree on the train data
train_data = train_data.sample(frac=1/10)
y_train = train_data["Label"]
X_train = train_data.drop(columns=["Label"])
X_train = X_train.select_dtypes(include='number')
del train_data
gc.collect()

0

# Arbol de decision

In [14]:
# Definir el modelo base con tu random_state
decision_tree_base = make_pipeline(SimpleImputer(), DecisionTreeClassifier(random_state=random_state))

# Definir los hiperparámetros que quieres probar
param_grid = {
    'decisiontreeclassifier__max_depth': [5, 8, 10, 15],
    'decisiontreeclassifier__min_samples_split': [2, 10, 20],
    'decisiontreeclassifier__min_samples_leaf': [1, 5, 10],
    'decisiontreeclassifier__criterion': ['gini', 'entropy']
}

# Crear el GridSearchCV
grid_search = GridSearchCV(decision_tree_base, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

# Ajustar el modelo con diferentes combinaciones
grid_search.fit(X_train, y_train)

# Ver los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
for param, value in grid_search.best_params_.items():
    param_clean = param.replace('decisiontreeclassifier__', '')
    print(f"{param_clean}: {value}")


# Calcular el AUC con el mejor modelo
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_train)[:, 1]
auc_dt = roc_auc_score(y_train, y_pred_proba)

# Imprimir el AUC
print(f"AUC: {auc_dt:.4f}")

Mejores hiperparámetros encontrados:
criterion: entropy
max_depth: 5
min_samples_leaf: 10
min_samples_split: 2
AUC: 0.8168


In [15]:
# Obtener los mejores hiperparámetros del grid search
best_params = grid_search.best_params_

# Crear el modelo con los mejores hiperparámetros encontrados
best_model = DecisionTreeClassifier(
    max_depth=best_params['decisiontreeclassifier__max_depth'],
    min_samples_split=best_params['decisiontreeclassifier__min_samples_split'],
    min_samples_leaf=best_params['decisiontreeclassifier__min_samples_leaf'],
    criterion=best_params['decisiontreeclassifier__criterion'],
    random_state=random_state
)

# Crear el pipeline con el mejor modelo
decision_tree = make_pipeline(SimpleImputer(), best_model)

# Entrenar el modelo con los datos de entrenamiento
decision_tree.fit(X_train, y_train)

# Predecir en el conjunto de evaluación
eval_data = eval_data.select_dtypes(include='number')
y_preds = decision_tree.predict_proba(eval_data.drop(columns=["id"]))[:, decision_tree.classes_ == 1].squeeze()

# Crear el archivo de envío
submission_df = pd.DataFrame({"id": eval_data["id"], "Label": y_preds})
submission_df["id"] = submission_df["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "decision_tree" y "auc_dt"
file_name = (
    f"decision_tree_predictions_{best_params['decisiontreeclassifier__max_depth']}_"
    f"{best_params['decisiontreeclassifier__min_samples_split']}_"
    f"{best_params['decisiontreeclassifier__min_samples_leaf']}_"
    f"{best_params['decisiontreeclassifier__criterion']}_"
    f"{auc_dt:.4f}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df.to_csv(os.path.join("submits", file_name), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name}")


Archivo guardado en: submits/decision_tree_predictions_5_2_10_entropy.csv


# Regresion Logistica

In [19]:
max_iter = 1000

In [20]:
# Definir el modelo base de regresión logística
logistic_model_base = LogisticRegression(random_state=random_state, max_iter=max_iter)

# Crear el pipeline
pipeline = make_pipeline(SimpleImputer(), logistic_model_base)

# Definir los hiperparámetros
param_grid = {
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [0.01, 0.1, 1, 10, 100],
    'logisticregression__solver': ['liblinear', 'saga'],
    'logisticregression__class_weight': [None, 'balanced']
}

# Crear GridSearchCV
grid_search_logistic = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search_logistic.fit(X_train, y_train)

# Ver los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
for param, value in grid_search_logistic.best_params_.items():
    param_clean = param.replace('logisticregression__', '')
    print(f"{param_clean}: {value}")


# Calcular el AUC con el mejor modelo
best_model = grid_search_logistic.best_estimator_
y_pred_proba = best_model.predict_proba(X_train)[:, 1]
auc_rl = roc_auc_score(y_train, y_pred_proba)

# Imprimir el AUC
print(f"AUC: {auc_rl:.4f}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/lucamazzarello_/opt/anaconda3/envs/TDVI/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

Mejores hiperparámetros encontrados:
logisticregression__C: 1
logisticregression__class_weight: balanced
logisticregression__penalty: l1
logisticregression__solver: liblinear
AUC: 0.7443


In [21]:
# Obtener los mejores hiperparámetros del grid search de regresión logística
best_params = grid_search_logistic.best_params_

# Crear el modelo con los mejores hiperparámetros encontrados
best_model = LogisticRegression(
    penalty=best_params['logisticregression__penalty'],
    C=best_params['logisticregression__C'],
    solver=best_params['logisticregression__solver'],
    class_weight=best_params['logisticregression__class_weight'],
    random_state=random_state,
    max_iter=max_iter
)

# Crear el pipeline con el mejor modelo
logistic_pipeline = make_pipeline(SimpleImputer(), best_model)

# Entrenar el modelo con los datos de entrenamiento
logistic_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de evaluación
eval_data = eval_data.select_dtypes(include='number')
y_preds_logistic = logistic_pipeline.predict_proba(eval_data.drop(columns=["id"]))[:, 1]

# Crear el archivo de envío
submission_df_logistic = pd.DataFrame({"id": eval_data["id"], "Label": y_preds_logistic})
submission_df_logistic["id"] = submission_df_logistic["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "logistic_regression"
file_name_logistic = (
    f"logistic_regression_predictions_{best_params['logisticregression__penalty']}_"
    f"C_{best_params['logisticregression__C']}_"
    f"solver_{best_params['logisticregression__solver']}_"
    f"class_weight_{best_params['logisticregression__class_weight']}.csv"
    f"{auc_rl:.4f}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_logistic.to_csv(os.path.join("submits", file_name_logistic), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_logistic}")


Archivo guardado en: submits/logistic_regression_predictions_l1_C_1_solver_liblinear_class_weight_balanced.csv


# Random Forest

In [2]:
# Definir el modelo base de Random Forest
rf_model = RandomForestClassifier(random_state=random_state)

# Crear el pipeline
pipeline_rf = make_pipeline(SimpleImputer(), rf_model)

# Definir los hiperparámetros
param_grid_rf = {
    'randomforestclassifier__n_estimators': [100, 200, 500],
    'randomforestclassifier__max_depth': [10, 20, None],
    'randomforestclassifier__min_samples_split': [2, 10, 20],
    'randomforestclassifier__min_samples_leaf': [1, 5, 10],
    'randomforestclassifier__class_weight': [None, 'balanced']
}

# Crear GridSearchCV
grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='roc_auc', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Ver los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
for param, value in grid_search_rf.best_params_.items():
    param_clean = param.replace('randomforestclassifier__', '')
    print(f"{param_clean}: {value}")


# Calcular el AUC con el mejor modelo
best_model = grid_search_rf.best_estimator_
y_pred_proba = best_model.predict_proba(X_train)[:, 1]
auc_rf = roc_auc_score(y_train, y_pred_proba)

# Imprimir el AUC
print(f"AUC: {auc_rf:.4f}")

NameError: name 'RandomForestClassifier' is not defined

In [1]:
# Obtener los mejores hiperparámetros del grid search de Random Forest
best_params = grid_search_rf.best_params_

# Crear el modelo con los mejores hiperparámetros encontrados
best_model_rf = RandomForestClassifier(
    n_estimators=best_params['randomforestclassifier__n_estimators'],
    max_depth=best_params['randomforestclassifier__max_depth'],
    min_samples_split=best_params['randomforestclassifier__min_samples_split'],
    min_samples_leaf=best_params['randomforestclassifier__min_samples_leaf'],
    class_weight=best_params['randomforestclassifier__class_weight'],
    random_state=random_state
)

# Crear el pipeline con el mejor modelo
rf_pipeline = make_pipeline(SimpleImputer(), best_model_rf)

# Entrenar el modelo con los datos de entrenamiento
rf_pipeline.fit(X_train, y_train)

# Predecir en el conjunto de evaluación
eval_data = eval_data.select_dtypes(include='number')
y_preds_rf = rf_pipeline.predict_proba(eval_data.drop(columns=["id"]))[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": eval_data["id"], "Label": y_preds_rf})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros de Random Forest
file_name_rf = (
    f"random_forest_predictions_n_estimators_{best_params['randomforestclassifier__n_estimators']}_"
    f"max_depth_{best_params['randomforestclassifier__max_depth']}_"
    f"min_samples_split_{best_params['randomforestclassifier__min_samples_split']}_"
    f"min_samples_leaf_{best_params['randomforestclassifier__min_samples_leaf']}_"
    f"class_weight_{best_params['randomforestclassifier__class_weight']}.csv"
    f"{auc_rf:.4f}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")


NameError: name 'grid_search_rf' is not defined